# K-Means Multi-Node Multi-GPU (MNMG) Demo

K-Means multi-Node multi-GPU implementation leverages Dask to spread data and computations across multiple workers. cuML uses One Process Per GPU (OPG) layout, which maps a single Dask worker to each GPU.

The main difference between cuML's MNMG implementation of k-means and the single-GPU is that the fit can be performed in parallel for each iteration, sharing only the centroids between iterations. The MNMG version also provides the same scalable k-means++ initialization algorithm as the single-GPU version.

Unlike the single-GPU implementation, The MNMG k-means API requires a Dask Dataframe or Array as input. `predict()` and `transform()` return the same type as input. The Dask cuDF Dataframe API is very similar to the Dask DataFrame API, but underlying Dataframes are cuDF, rather than Pandas. Dask cuPy arrays are also available.

For information about cuDF, refer to the [cuDF documentation](https://docs.rapids.ai/api/cudf/stable).

For additional information on cuML's k-means implementation: 
https://docs.rapids.ai/api/cuml/stable/api.html#cuml.dask.cluster.KMeans.

## Imports

In [1]:
from cuml.dask.cluster.kmeans import KMeans as cuKMeans
from cuml.dask.common import to_dask_df
from cuml.dask.datasets import make_blobs
from cuml.metrics import adjusted_rand_score
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask_ml.cluster import KMeans as skKMeans
import cupy as cp

## Start Dask Cluster

We can use the `LocalCUDACluster` to start a Dask cluster on a single machine with one worker mapped to each GPU. This is called one-process-per-GPU (OPG). 

In [2]:
cluster = LocalCUDACluster(threads_per_worker=1)
client = Client(cluster)

/opt/conda/envs/rapids/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39581 instead
  warnings.warn(
2023-10-04 21:26:22,562 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-10-04 21:26:22,562 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


## Define Parameters

In [3]:
n_samples = 1000000
n_features = 2

n_total_partitions = len(list(client.has_what().keys()))

## Generate Data

### Device

We can generate a Dask cuPY Array of synthetic data for multiple clusters using `cuml.dask.datasets.make_blobs`.

In [4]:
X_dca, Y_dca = make_blobs(n_samples, 
                          n_features,
                          centers = 5, 
                          n_parts = n_total_partitions,
                          cluster_std=0.1, 
                          verbose=True)

### Host

We collect the Dask cuPy Array on a single node as a cuPy array. Then we transfer the cuPy array from device to host memory into a Numpy array.

In [5]:
X_cp = X_dca.compute()
X_np = cp.asnumpy(X_cp)
del X_cp

## Scikit-learn model

### Fit and predict

Since a scikit-learn equivalent to the multi-node multi-GPU K-means in cuML doesn't exist, we will use Dask-ML's implementation for comparison.

In [6]:
%%time
kmeans_sk = skKMeans(init="k-means||",
                     n_clusters=5,
                     n_jobs=-1,
                     random_state=100)

kmeans_sk.fit(X_np)

/opt/conda/envs/rapids/lib/python3.10/site-packages/dask/base.py:1368: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


CPU times: user 6.13 s, sys: 559 ms, total: 6.69 s
Wall time: 26.9 s


KMeans(n_clusters=5, n_jobs=-1, random_state=100)

In [7]:
%%time
labels_sk = kmeans_sk.predict(X_np).compute()

CPU times: user 212 ms, sys: 21 ms, total: 233 ms
Wall time: 797 ms


## cuML Model

### Fit and predict

In [8]:
%%time
kmeans_cuml = cuKMeans(init="k-means||",
                       n_clusters=5,
                       random_state=100)

kmeans_cuml.fit(X_dca)

CPU times: user 55.2 ms, sys: 3.51 ms, total: 58.7 ms
Wall time: 1.45 s


In [9]:
%%time
labels_cuml = kmeans_cuml.predict(X_dca).compute()

CPU times: user 22.5 ms, sys: 0 ns, total: 22.5 ms
Wall time: 45 ms


## Compare Results

In [10]:
score = adjusted_rand_score(labels_sk, labels_cuml)

In [11]:
passed = score == 1.0
print('compare kmeans: cuml vs sklearn labels_ are ' + ('equal' if passed else 'NOT equal'))

compare kmeans: cuml vs sklearn labels_ are equal
